In [1]:
! pip install PyMySQL

     |████████████████████████████████| 43 kB 1.1 MB/s 


In [2]:
import pymysql.cursors
# 데이터베이스 연결
connection = pymysql.connect(host='13.124.77.49', #EC2 아이피
                             user='USER_SQLSTK',  #데이터베이스 사용자
                             password='1qaz2wsx', #데이터베이스 암호
                             database='DB_SQLSTK', #데이터베이스 이름
                             charset='utf8',       # 언어 설정 
                             cursorclass=pymysql.cursors.DictCursor # 조회결과를 dictionary로 리턴
                             )

OperationalError: ignored

In [ ]:
cursor = connection.cursor()
cursor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 실행할 SQL 쿼리
sql = """
      SELECT T1.STK_CD, T1.STK_NM, T1.SEC_NM
              ,T2.DT, O_PRC, T2.L_PRC, T2.H_PRC, T2.C_PRC, T2.VOL
              ,M5_PRC , M10_PRC, M20_PRC
      FROM stock T1
      INNER JOIN HISTORY_DT T2
      ON (T2.STK_CD = T1.STK_CD)
      WHERE T1.STK_NM = '성신양회'
      AND T2.DT >= '2019-01-01'
      AND T2.DT <= '2019-02-28';
"""  

# SQL 쿼리를 실행할 준비를 함
cursor.execute(sql)

In [ ]:
# SQL 쿼리를 실행하고 결과를 result에 저장
result = cursor.fetchall()
# SQL 쿼리 실행 결과 출력
result

In [ ]:
import pandas as pd
# result를 DataFrame으로 변환
df = pd.DataFrame(result)
df

In [ ]:
# df의 컬럼타입 조회
df.info()

In [ ]:
# DT 컬럼을 날짜와 시간 타입으로 변환
df["DT"] = pd.to_datetime(df["DT"])
df.info()

In [ ]:
# O_PRC 컬럼부터 마지막 컬럼까지 정수로 변환
df.loc[: , "O_PRC":] = df.loc[: , "O_PRC":].astype(int)
df.info()

In [ ]:
import plotly.graph_objects as go

# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

ch = go.Figure(data=candle) # 봉차트 candle을 그릴 준비를 함
ch.show() # 봉차트 출력

In [ ]:
import plotly.subplots as ms

# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

In [ ]:
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# Samsung stock price 제목 출력
fig.update_layout( title='성신양회' )

In [ ]:
import pandas as pd
# from datetime import datetime
df1 = pd.read_csv("sungshin_m60_prc.csv", engine="python") #encoding="cp949"
# df1['dt'] = pd.to_datetime(df1["dt"])
df1['60days'] = df1['c_prc']/df1['M60_PRC'] * 100 # 60일 이평선의 이격도
df['M60_PRC'] = df1['M60_PRC']
df['20days'] = df['C_PRC']/df['M20_PRC']*100 # 20일 이평선의 이격도
df['60days'] = df1['60days']
df1

In [ ]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

# 5일 이동 평균 선 차트 
ma5 = go.Scatter(x=df.DT, y=df['M5_PRC'], line=dict(color='black', width=0.8), name='ma5')
# 10일 이동 평균 선 차트 
ma10 = go.Scatter(x=df.DT, y=df['M10_PRC'], line=dict(color='red', width=0.9), name='ma20')
# 20일 이동 평균 선 차트 
ma20 = go.Scatter(x=df.DT, y=df['M20_PRC'], line=dict(color='green', width=1), name='ma60')

# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma5 차트 출력
fig.add_trace(ma5, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma10 차트 출력
fig.add_trace(ma10, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma20 차트 출력
fig.add_trace(ma20, row=1, col=1)

# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# Samsung stock price 제목 출력
fig.update_layout( title='성신양회' )

